In [9]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from torchvision import datasets
from pathlib import Path
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = 'tight'
from torchvision.transforms import v2
from torchvision.io import read_image
torch.manual_seed(1) # seed
from helpers import plot, random_combination
import os
import shutil
import time
from tempfile import TemporaryDirectory
from PIL import Image
from os import listdir
from os.path import isfile, join

from torchvision.models import alexnet, AlexNet_Weights, convnext_tiny, ConvNeXt_Tiny_Weights, swin_v2_t, Swin_V2_T_Weights, efficientnet_v2_s, EfficientNet_V2_S_Weights, mobilenet_v3_large
from torchvision.models import MobileNet_V3_Large_Weights


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights = None) # weights = weights
standard_preprocess = weights.transforms()

In [12]:
data_transforms = {
                    'train': v2.Compose([
                            v2.RandomApply([v2.RandAugment()], p = .5),
                            v2.RandomResizedCrop(size=(224, 224), antialias=True),
                            v2.RandomHorizontalFlip(p=0.5),
                            v2.ToTensor(),
                            v2.ToDtype(torch.float32, scale=True),
                            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
                   'val': standard_preprocess}
data_dir = './Dataset_Categorised_with_Supplement/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

c:\Users\zoga\anaconda3\envs\kaggle_comp\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [13]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

def visualize_model_predictions(model,img_path):
    was_training = model.training
    model.eval()

    img = Image.open(img_path)
    img = standard_preprocess(img)
    img = img.unsqueeze(0)
    img = img.to(device)

    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)

        ax = plt.subplot(2,2,1)
        ax.axis('off')
        ax.set_title(f'Predicted: {class_names[preds[0]]}')
        imshow(img.cpu().data[0])

        model.train(mode=was_training)

def predict(model,img_path):
    was_training = model.training
    model.eval()

    img = Image.open(img_path)
    img = data_transforms['val'](img)
    img = img.unsqueeze(0)
    img = img.to(device)

    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)

        return preds
    
def predict_proba(model,img_path):
    was_training = model.training
    model.eval()

    img = Image.open(img_path)
    img = data_transforms['val'](img)
    img = img.unsqueeze(0)
    img = img.to(device)

    with torch.no_grad():
        outputs = model(img)
        preds = outputs

        return preds

In [14]:
df_train_labels = pd.read_csv('./Dataset/train_labels.csv')

In [15]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    epoch_data = {'train_acc': [], 'train_loss': [], 'val_acc': [], 'val_loss': []}

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                if phase == 'train':
                    epoch_data['train_acc'].append(epoch_acc)
                    epoch_data['train_loss'].append(epoch_loss)
                elif phase == 'val':
                    epoch_data['val_acc'].append(epoch_acc)
                    epoch_data['val_loss'].append(epoch_loss)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
        
    return model, epoch_data, best_acc

In [16]:
all_test_files_jpeg = listdir('./Dataset/test/')
all_test_files = ['./Dataset/test/' + i for i in all_test_files_jpeg]

model_architectures = [alexnet, convnext_tiny, swin_v2_t, efficientnet_v2_s, mobilenet_v3_large]
model_names = ['alex', 'convnext', 'swin', 'efficientnet', 'mobilenet']

for model_name, model_a in zip(model_names, model_architectures):

    print(f'\n{model_name}')

    model_temp = model_a(weights = None)

    model_classifier_temp = nn.Sequential(model_temp, nn.Linear(1000, 4))
    model_classifier_temp.to(device)

    optimizer_ft = optim.Adam(model_classifier_temp.parameters(), lr=1e-4)
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.9)
    criterion = nn.CrossEntropyLoss()

    model_ft_temp, model_ft_epoch_data_temp, model_best_acc_temp = train_model(model_classifier_temp, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=200)
    
    print('performing predictions...')

    all_test_pred = np.empty((0,4))
    for file_str in all_test_files:
        all_test_pred = np.vstack([all_test_pred, predict_proba(model_ft_temp, file_str).cpu().numpy()])

    # saving...
    
    save_model_loc_temp = f'./models/{model_name}_{model_best_acc_temp:.2f}.pt'
    save_results_loc_temp = f'./predictions/{model_name}_{model_best_acc_temp:.2f}_results.npy'

    print(f'saving model to {save_model_loc_temp}...')
    torch.save(model_ft_temp.state_dict(), save_model_loc_temp)

    print(f'saving results to {save_results_loc_temp}...')
    with open(save_results_loc_temp, 'wb') as f:
        np.save(save_results_loc_temp, all_test_pred)
    
    


alex
Epoch 0/199
----------
train Loss: 1.3415 Acc: 0.3086
val Loss: 1.4144 Acc: 0.3827

Epoch 1/199
----------
train Loss: 1.3175 Acc: 0.3593
val Loss: 1.3628 Acc: 0.3704

Epoch 2/199
----------
train Loss: 1.2658 Acc: 0.4193
val Loss: 1.2057 Acc: 0.4938

Epoch 3/199
----------
train Loss: 1.1947 Acc: 0.5012
val Loss: 1.1238 Acc: 0.5309

Epoch 4/199
----------
train Loss: 1.1456 Acc: 0.5360
val Loss: 1.1040 Acc: 0.6173

Epoch 5/199
----------
train Loss: 1.1153 Acc: 0.5490
val Loss: 1.0957 Acc: 0.5679

Epoch 6/199
----------
train Loss: 1.0848 Acc: 0.5608
val Loss: 0.9814 Acc: 0.5556

Epoch 7/199
----------
train Loss: 1.0665 Acc: 0.5749
val Loss: 0.9855 Acc: 0.6543

Epoch 8/199
----------
train Loss: 1.0346 Acc: 0.5884
val Loss: 0.9847 Acc: 0.6543

Epoch 9/199
----------
train Loss: 0.9961 Acc: 0.6182
val Loss: 0.9020 Acc: 0.6790

Epoch 10/199
----------
train Loss: 0.9725 Acc: 0.6230
val Loss: 0.9704 Acc: 0.6296

Epoch 11/199
----------
train Loss: 0.9693 Acc: 0.6213
val Loss: 0.72